# 8.4.5 선형회귀 분석 실습

##### 보스턴 집값 데이터 셋을 이용하여 집값을 예측하는 모형을 만들어본다.

##### **릿지 회귀 분석**, **라쏘 회귀 분석**, **엘라스틱 넷**을 사용한 결과도 비교한다.
---



# Dataset import
## 데이터 불러오기

In [2]:
import pandas as pd
import numpy as np

In [5]:
# pandas, numpy 불러오지 않고 쓰면 긴 메세지가 뜨는 것을 확인할 수 있었다.

from sklearn import datasets
raw_boston = datasets.load_boston()

In [6]:
# 데이터 셋 내 피처 살펴보기
raw_boston.feature_names

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

# 피처, 타깃 데이터 지정

In [9]:
X = raw_boston.data
y = raw_boston.target

## 트레이닝, 테스트 데이터 분할

In [13]:
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 1)

## 데이터 표준화

In [15]:
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
std_scale.fit(X_tn)                   # X 트레이닝 데이터 기준으로 std scaler fitting
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

# 선형 회귀 분석 모델

먼저 선형 회귀 모델을 만들어 예측해보도록 한다.

## 데이터 학습

In [16]:
from sklearn.linear_model import LinearRegression
clf_lr = LinearRegression()    # 사용할 모델 이름 classifier _ linear regression
clf_lr.fit(X_tn_std, y_tn)     # 피팅

LinearRegression()

## 선형 회귀 모델의 계수, 상수항 확인

선형회귀는 $ y = wx +b $ 형태로 볼 수 있었다!

모형 적합 후 coef_ method를 활용하여 추정된 회귀 계수(w)를 확인해 볼 수 있다.

또한 intercept_ method를 활용하여 추정된 상수항($b$)을 확인해 볼 수 있다. (intercept라니 이름 참 잘지은 것 같다.)

In [17]:
# 회귀 계수 확인

print(clf_lr.coef_)

[-1.07145146  1.34036243  0.26298069  0.66554537 -2.49842551  1.97524314
  0.19516605 -3.14274974  2.66736136 -1.80685572 -2.13034748  0.56172933
 -4.03223518]


In [18]:
# 상수항

print(clf_lr.intercept_)

22.344591029023768


# L2제약식을 적용한 릿지 회귀 분석 모델

기본 선형 회귀 분석에 L2 제약식을 적용한 **릿지 회귀 분석**을 실습해보자.

## 데이터 학습

In [19]:
from sklearn.linear_model import Ridge    # Ridge Regression
clf_ridge = Ridge(alpha = 1)              # 기본은 1, 반드시 양수여야하고 값이 클 수록 강한 제약식.
clf_ridge.fit(X_tn_std, y_tn)

Ridge(alpha=1)

## 릿지 회귀 모델의 계수, 상수항 확인

In [20]:
# 회귀 계수 확인

print(clf_ridge.coef_)

[-1.05933451  1.31050717  0.23022789  0.66955241 -2.45607567  1.99086611
  0.18119169 -3.09919804  2.56480813 -1.71116799 -2.12002592  0.56264409
 -4.00942448]


In [21]:
# 상수항

print(clf_ridge.intercept_)

22.344591029023768


신기하게도 거의 비슷한따 상수항은 인간 입장에서는 일치한다고 봐도 무방할 정도이다.

# L1제약식을 적용한 라쏘 회귀 분석 모델

이번엔 기본 선형 회귀 분석에 L1제약식을 적용한 **라쏘 회귀 분석**을 실습해보자.

## 데이터 학습

In [22]:
from sklearn.linear_model import Lasso    # Lasso Regression
clf_lasso = Lasso(alpha = 0.01)           # 기본은 1, 릿지와 마찬가지로 제약의 정도를 나타냄
clf_lasso.fit(X_tn_std, y_tn)

Lasso(alpha=0.01)

## 라쏘 회귀 모델의 계수, 상수항 확인

In [23]:
# 계수 확인

print(clf_lasso.coef_)

[-1.04326518  1.27752711  0.1674367   0.66758228 -2.41559964  1.99244179
  0.14733958 -3.09473711  2.46431135 -1.60552274 -2.11046422  0.55200229
 -4.00809905]


In [25]:
# 상수항 확인

print(clf_lasso.intercept_)

22.344591029023768


계수는 조금씩 차이가 나지만 상수항은 같게 나왔다.

# ElasticNet 모델

이번에는 엘라스틱 넷을 사용해보도록 한다.

## 데이터 학습

In [30]:
from sklearn.linear_model import ElasticNet
clf_elastic = ElasticNet(alpha = 0.01, l1_ratio = 0.01) # alpha는 L1제약식의 크기와 L2제약식의 크기를 합한 전체 제약식의 크기
                                                        # l1_ratio는 전체 제약의 크기인 alpha 값에서 L1제약이 차지하는 비율을 의미
                                                        # l1_ratio=0이면 L1제약없이 L2제약만 적용되는 릿지 회귀분석을 의미
clf_elastic.fit(X_tn_std, y_tn)

ElasticNet(alpha=0.01, l1_ratio=0.01)

## 엘라스틱넷 계수, 상수항 확인

In [31]:
# 계수 확인

print(clf_elastic.coef_)

[-1.02916603  1.23681955  0.15236504  0.67859622 -2.34646781  2.02965524
  0.14575132 -2.98592423  2.32013379 -1.48829485 -2.09271972  0.56506801
 -3.9495281 ]


In [32]:
# 상수항 확인

print(clf_elastic.intercept_)

22.344591029023768


# 데이터 예측
만들어본 기본 선형회귀, 릿지 회귀, 라쏘 회귀, 엘라스틱넷 회귀 모델로 데이터 predict를 진행해본다.

## 데이터 예측

In [34]:
pred_lr = clf_lr.predict(X_te_std)
pred_ridge = clf_ridge.predict(X_te_std)
pred_lasso = clf_lasso.predict(X_te_std)
pred_elastic = clf_elastic.predict(X_te_std)

## 모형평가 :  R제곱값

R제곱값으로 각 모형들을 평가해본다. 0과 1사이값을 가지며, 높을수록 좋은 성능을 의미한다.

In [36]:
# import library
from sklearn.metrics import r2_score as r2

# each model test
print("Linear regression: ", r2(y_te, pred_lr))
print("Ridge regression: ", r2(y_te, pred_ridge))
print("Lasso regression: ", r2(y_te, pred_lasso))
print("ElasticNet regression: ", r2(y_te, pred_elastic))

Linear regression:  0.7789410172622859
Ridge regression:  0.7789704562726603
Lasso regression:  0.7787621490259895
ElasticNet regression:  0.7787876079239252


R제곱값에서는 근소한 차이로 릿지 회귀 모형이 좋은 성능을 보인 것으로 나왔으나, 정말 큰 차이가 나지 않는다.

## 모형평가 : MSE(mean squared error! 오차 제곱 합의 평균)

이번에는 mse로 평가해보도록 한다. **'오차'**의 제곱의 합의 평균이므로 작을 수록 좋은 성능을 의미한다.

In [37]:
# import library
from sklearn.metrics import mean_squared_error as mse

# each model test
print("Linear regression: ", mse(y_te, pred_lr))
print("Ridge regression: ", mse(y_te, pred_ridge))
print("Lasso regression: ", mse(y_te, pred_lasso))
print("ElasticNet regression: ", mse(y_te, pred_elastic))

Linear regression:  21.897765396049483
Ridge regression:  21.894849212618773
Lasso regression:  21.915483810504824
ElasticNet regression:  21.91296189093687


근소한 차이로 릿지 회귀 모형이 좋은 성능을 보이는 것으로 나왔다. (뭐 물론 어떤 테스트를 하던 테스트 값에 근소하면 근소할 수록 잘 나오는 건 당연하겠지만.)

# 궁금해요실험: No Standardization
---

데이터 표준화를 하지 않고 그냥 했을 때의 결과는 많이 차이가 날까? 궁금해서 진행하는 부분.

### 트레이닝/테스트 데이터 분할

부터 다시 해볼 것이에요 왜냐하면 바보라서 까먹을 수 있으니 코드를 눈에 박기 위해서에요.

In [38]:
from sklearn.model_selection import train_test_split                # 데이터 분할을 위한 함수
# from sklearn.preprocessing import StandardScaler                  # 데이터 표준화를 위한 함수

X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0) # 데이터 분할

# std_scale = StandardScaler()                                      # 표준화 스케일러 선언
# std_scale.fit(X_tn)                                               # 트레이닝 피처(데이터)를 기준으로 표준화 스케일러 적합

# X_tn_std = std_scale.transform(X_tn)                              # 트레인, 테스트 데이터를 표준화 스케일러 처리
# X_te_std = std_scale.transform(X_te)

## 각 모델에 적합 및 예측

### 데이터 학습

In [39]:
# 각 선형 회귀 모형 import
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

# 모형 선언 (위쪽에서 했었던 옵션 그대로 유지)
clf_lr_nstd = LinearRegression()
clf_ridge_nstd = Ridge(alpha = 1)
clf_lasso_nstd = Lasso(alpha = 0.01)
clf_elast_nstd = ElasticNet(alpha = 0.01, l1_ratio = 0.01)

# 데이터 적합
clf_lr_nstd.fit(X_tn, y_tn)
clf_ridge_nstd.fit(X_tn, y_tn)
clf_lasso_nstd.fit(X_tn, y_tn)
clf_elast_nstd.fit(X_tn, y_tn)

ElasticNet(alpha=0.01, l1_ratio=0.01)

### 데이터 예측

In [41]:
pred_lr_nstd = clf_lr_nstd.predict(X_te)
pred_ridge_nstd = clf_lasso_nstd.predict(X_te)
pred_lasso_nstd = clf_ridge_nstd.predict(X_te)
pred_elast_nstd = clf_elast_nstd.predict(X_te)

### 계수 및 상수항 확인

확인을 한다고 해서 크게 뭔가를 알 수 있는 것은 아니지만 전자두뇌가 뱉은 일련의 숫자를 보는 것은 마음이 편안하다.

이때는 각각의 예측 방식이 표준화가 된 데이터로 학습했을 때의 모델을 가지고 만든 것을 비교하도록 한다.

In [44]:
# 일반 선형회귀 모델 계수 및 상수항 확인

print('계수 결과 비교')
print(clf_lr.coef_)
print(clf_lr_nstd.coef_)

print('상수항 결과 비교')
print(clf_lr.intercept_)
print(clf_lr_nstd.intercept_)

계수 결과 비교
[-1.07145146  1.34036243  0.26298069  0.66554537 -2.49842551  1.97524314
  0.19516605 -3.14274974  2.66736136 -1.80685572 -2.13034748  0.56172933
 -4.03223518]
[-1.17735289e-01  4.40174969e-02 -5.76814314e-03  2.39341594e+00
 -1.55894211e+01  3.76896770e+00 -7.03517828e-03 -1.43495641e+00
  2.40081086e-01 -1.12972810e-02 -9.85546732e-01  8.44443453e-03
 -4.99116797e-01]
상수항 결과 비교
22.344591029023768
36.93325545712031


벌써 결과가 다르게 나온 부분들을 볼 수 있는게 재미있다. 일단 해당 데이터가 집값이다보니까 데이터 스케일때문인지 계수들이 지저분(?) 한 것을 확인할 수 있었다.

In [45]:
# 릿지 회귀 모델 계수 및 상수항 확인

print('계수 결과 비교')
print(clf_ridge.coef_)
print(clf_ridge_nstd.coef_)

print('상수항 결과 비교')
print(clf_ridge.intercept_)
print(clf_ridge_nstd.intercept_)

계수 결과 비교
[-1.05933451  1.31050717  0.23022789  0.66955241 -2.45607567  1.99086611
  0.18119169 -3.09919804  2.56480813 -1.71116799 -2.12002592  0.56264409
 -4.00942448]
[-0.11478591  0.04541838 -0.03540308  2.30329133 -8.12744537  3.80634143
 -0.01428219 -1.33693873  0.21674809 -0.01175498 -0.90393728  0.00881273
 -0.50742473]
상수항 결과 비교
22.344591029023768
31.849731779351067


In [47]:
# 라쏘 회귀 모델 계수 및 상수항 확인

print('계수 결과 비교')
print(clf_lasso.coef_)
print(clf_lasso_nstd.coef_)

print('상수항 결과 비교')
print(clf_lasso.intercept_)
print(clf_lasso_nstd.intercept_)

계수 결과 비교
[-1.04326518  1.27752711  0.1674367   0.66758228 -2.41559964  1.99244179
  0.14733958 -3.09473711  2.46431135 -1.60552274 -2.11046422  0.55200229
 -4.00809905]
[-1.16104087e-01  4.46248964e-02 -1.83063803e-02  2.23355680e+00
 -1.20664164e+01  3.77492436e+00 -1.01096425e-02 -1.38336756e+00
  2.29616121e-01 -1.15665955e-02 -9.47538722e-01  8.63409535e-03
 -5.04338626e-01]
상수항 결과 비교
22.344591029023768
34.591050491203546


In [48]:
# 엘라스틱넷 회귀 모델 계수 및 상수항 확인

print('계수 결과 비교')
print(clf_elastic.coef_)
print(clf_elast_nstd.coef_)

print('상수항 결과 비교')
print(clf_elastic.intercept_)
print(clf_elast_nstd.intercept_)

계수 결과 비교
[-1.02916603  1.23681955  0.15236504  0.67859622 -2.34646781  2.02965524
  0.14575132 -2.98592423  2.32013379 -1.48829485 -2.09271972  0.56506801
 -3.9495281 ]
[-0.11307276  0.04665313 -0.05344769  2.07096238 -3.50564722  3.75042554
 -0.01808508 -1.27742464  0.20563815 -0.01218142 -0.85806729  0.00903266
 -0.51843859]
상수항 결과 비교
22.344591029023768
29.35106342597792


계수들의 모양이 일정하지 않고 모형마다 조금씩 다른데, 그나마 릿지 회귀와 엘라스틱넷 회귀가 모양이 표준화때와 비슷한 것 같다.

## 모형 평가

위쪽에서 예측은 했으므로 모형을 평가해보도록 한다.

### 모형평가 :  R제곱값

서로를 비교한다기보다 표준화 전과 비교하는 것이 우선이므로 위쪽에 있었던 결과를 그대로 함께 가져와 비교해본다.

In [53]:
# import library
# from sklearn.metrics import r2_score as r2

# each model test
print("표준화 후 데이터 셋의 R제곱값")
print("Linear regression: ", r2(y_te, pred_lr))
print("Ridge regression: ", r2(y_te, pred_ridge))
print("Lasso regression: ", r2(y_te, pred_lasso))
print("ElasticNet regression: ", r2(y_te, pred_elastic))

표준화 후 데이터 셋의 R제곱값
Linear regression:  -0.41653241924318585
Ridge regression:  -0.4121509210377161
Lasso regression:  -0.4092744818108047
ElasticNet regression:  -0.4012866165843114


In [52]:
print("표준화 전 데이터 셋의 R제곱값")
print("Linear regression: ", r2(y_te, pred_lr_nstd))
print("Ridge regression: ", r2(y_te, pred_ridge_nstd))
print("Lasso regression: ", r2(y_te, pred_lasso_nstd))
print("ElasticNet regression: ", r2(y_te, pred_elast_nstd))

표준화 전 데이터 셋의 R제곱값
Linear regression:  0.635463843320211
Ridge regression:  0.6316839365596243
Lasso regression:  0.626618220461385
ElasticNet regression:  0.6183010891242328


큰 값이 더 좋은 것으로 나타난다. 전체적으로 표준화 전 데이터 셋의 R제곱값이 더 작다.

### 모형평가 : MSE(mean squared error! 오차 제곱 합의 평균)


In [54]:
# import library
# from sklearn.metrics import mean_squared_error as mse

# each model test
print("표준화 후 데이터 셋의 MSE")
print("Linear regression: ", mse(y_te, pred_lr))
print("Ridge regression: ", mse(y_te, pred_ridge))
print("Lasso regression: ", mse(y_te, pred_lasso))
print("ElasticNet regression: ", mse(y_te, pred_elastic))

표준화 후 데이터 셋의 MSE
Linear regression:  115.72930398822027
Ridge regression:  115.37133989868988
Lasso regression:  115.13633764588388
ElasticNet regression:  114.48373692142769


In [56]:
# import library
# from sklearn.metrics import mean_squared_error as mse

# each model test
print("표준화 후 데이터 셋의 MSE")
print("Linear regression: ", mse(y_te, pred_lr_nstd))
print("Ridge regression: ", mse(y_te, pred_ridge_nstd))
print("Lasso regression: ", mse(y_te, pred_lasso_nstd))
print("ElasticNet regression: ", mse(y_te, pred_elast_nstd))

표준화 후 데이터 셋의 MSE
Linear regression:  29.78224509230252
Ridge regression:  30.091059753090008
Lasso regression:  30.504923770804737
ElasticNet regression:  31.184425212318875


아진짜 왜그러는거야